In [ ]:
!pip install sympy

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import networkx as nx

from sklearn.datasets import make_circles

import sys 
sys.path.append("../seminar1/")
from simplicialx.simplicial import SimplicialComplex

from itertools import permutations
from sympy.combinatorics.permutations import Permutation

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from tqdm import tqdm

In [ ]:
def neighbors_lower(x_id, A):
    LT = np.tril(A)
    return list(np.nonzero(LT[x_id])[0])

def intersect(d):
    return list(set(d[0]).intersection(*d))

def grade(K):
    dim = len(max(K, key=len))
    K_graded = [[] for _ in range(dim)]
    for sigma in K:
        dim_sigma = len(sigma) - 1
        #print("{}-dim simplex: {}".format(dim_sigma, sigma))
        if dim_sigma == 0:
            sigma = sigma[0]
        K_graded[dim_sigma].append(sigma)
        
    for k, item in enumerate(K_graded):
        if k==0:
            item_array = np.expand_dims(np.array(item), 1)
        else:
            item_array = np.array(item)
        
        K_graded[k] = item_array

    return K_graded

def expand_incremental(G, k=2):
    
    k = k+1 # k-clique instead of (k-1)-simplex
    
    def add_cofaces(A, k, tau, N_lower, simplices):
        
        # V = V \cup tau
        simplices.append(tau)
        
        # if dim(tau) >= k
        if len(tau) >= k:
            return
        else:
            # foreach v \in N
            for v in N_lower:
                
                # \sigma = \tau \cup v
                sigma = sorted(tau + [v])
                M = intersect([N_lower, neighbors_lower(v, A)])
                add_cofaces(A, k, sigma, M, simplices)
            
        
        return simplices
    
    simplices = []
    A = nx.to_numpy_array(G)

    # define vertex set
    V = [item for item in nx.nodes(G)]
    
    for u in V:
        N_lower = neighbors_lower(u, A)
        add_cofaces(A, k, [u], N_lower, simplices)
    
    return grade(simplices)

def boundary_matrix(K, k=1):
    
    # get simplices, lists of arrays
    sigmas, taus = K[k], K[k-1]
    
    # init boundary matrix B
    B = np.zeros((len(taus), len(sigmas)))
    
    # iterate over columns of B, i.e. k-simplices sigmas
    for j, sigma in enumerate(sigmas):
        
        # omit h-th vertex in a k-simplex sigma
        for h in range(k+1):
            
            idx_h = np.ones(k+1).astype(bool)
            idx_h[h] = False
            
            # get k-1 simplex tau and find i of it
            tau = sigma[idx_h]
            i = np.flatnonzero(np.equal(taus, tau).all(axis=1))[0]
            
            B[i,j] = (-1) ** h # orientation sign
            
    return B

def generalized_boundary_matrix(K, k=1, p=1, eta_sign=1):
    
    # get simplices, lists of arrays
    sigmas, taus = K[k], K[k-p]
    
    # init boundary matrix B
    B = np.zeros((len(taus), len(sigmas)))
    
    # iterate over columns of B, i.e. k-simplices sigmas
    for j, sigma in enumerate(sigmas):
        
        mask = np.ones(k+1).astype(bool)
        mask[:p] = False

        # omit h_1,...,h_j vertices in a k-simplex sigma
        for idx in np.array(sorted(set(permutations(mask)))):

            # get k-p simplex tau and find i of it
            tau = sigma[idx]
            i = np.flatnonzero(np.equal(taus, tau).all(axis=1))[0]

            vertices_in = list(np.array(range(k+1))[idx])
            vertices_out = list(np.array(range(k+1))[~idx])

            B[i,j] = eta_sign * Permutation(vertices_out + vertices_in).signature() # orientation sign
            
    return B

def normalize(L):
    n = L.shape[0]
    D = np.diag(np.diag(L)) + np.eye(n) * 1e-9
    D_inv = np.linalg.inv(D)
    return D_inv @ L

### Hodge Laplacian for harmonic representatives

In [ ]:
X = np.array([
    [1, 3], # 
    [3, 3],
    [0, 2],
    [2, 2],
    [4, 2],
    [0, 0],
    [4, 0]
])

In [ ]:
cmplx = SimplicialComplex()

cmplx.add([0,1])
cmplx.add([0,2,3])
cmplx.add([1,3,4])
cmplx.add([2,5])
cmplx.add([4,6])
cmplx.add([5,6])

cmplx.simplices[0]

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(X[:,0], X[:,1], c="b", s=25)
plt.xlim(-0.5, 4.5)
plt.ylim(-0.5, 3.5)

# edges
for edge in cmplx.simplices[1]:
    plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.25, linewidth=1)

# triangles
for triangle in cmplx.simplices[2]:
    t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
    plt.gca().add_patch(t)
    
for vertex in cmplx.simplices[0]:
        plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.15), fontsize=12)

plt.show()

#### Boundary matrices

In [ ]:
B1 = cmplx.boundary_operator_matrix(k=1) # edge-vertices
B2 = cmplx.boundary_operator_matrix(k=2) # triangle-edges

In [ ]:
B1

In [ ]:
B2

#### Graph Laplacian

In [ ]:
L0 = B1 @ B1.T
L0

In [ ]:
np.linalg.eigvalsh(L0)

#### Hodge Laplacian

In [ ]:
L1 = B1.T @ B1 + B2 @ B2.T
L1

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(L1)
eigenvalues

In [ ]:
np.abs(eigenvectors[:,0])

In [ ]:
np.abs(eigenvectors[:,1]) / np.abs(eigenvectors[:,1]).max()

In [ ]:
# for each nonzero eigenvalue
for k in range(np.isclose(eigenvalues, 0).sum()):

    plt.figure(figsize=(6,6))
    plt.scatter(X[:,0], X[:,1], c="b", s=25)
    plt.xlim(-0.5, 4.5)
    plt.ylim(-0.5, 3.5)
    
    # edges
    for edge in cmplx.simplices[1]:
        plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=0.05, linewidth=1)

    # triangles
    for triangle in cmplx.simplices[2]:
        t = plt.Polygon(X[triangle], color="b", alpha=0.1, linewidth=0)
        plt.gca().add_patch(t)

    # harmonic representatives
    eigenvector_normalized = np.abs(eigenvectors[:,k]) / np.abs(eigenvectors[:,k]).max()
    for i, edge in enumerate(cmplx.simplices[1]):
        plt.plot(X[edge][:,0], X[edge][:,1], c="b", alpha=eigenvector_normalized[i], linewidth=5*eigenvector_normalized[i], label="({}, {}): {:.3f}".format(edge[0], edge[1], np.abs(eigenvectors[i,k])))

    for vertex in cmplx.simplices[0]:
        plt.annotate(vertex[0], (X[vertex,0]-0.2, X[vertex,1]-0.15), fontsize=12)

    plt.legend(loc="upper right", bbox_to_anchor=(1.35, 1.0))
    plt.show()


### Hodge Laplacian for classification

In [ ]:
n, m, ps = 50, 10, [0.075, 0.1, 0.15]

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16.5,5), dpi=200)

def stack(idx):
    ret = np.empty((0, 2))
    for _id in idx:
        ret = np.vstack((ret, X[_id,:]))
    return ret

for i, p_i in enumerate(ps):
    
    G = nx.generators.random_graphs.watts_strogatz_graph(n, m, p_i)
    K = expand_incremental(G, k=2)
    
    # boundary matrices
    B1 = boundary_matrix(K, k=1)
    B2 = boundary_matrix(K, k=2)
    
    # Laplacians
    L0 = B1 @ B1.T
    L1 = B1.T @ B1 + B2 @ B2.T
    
    # eigenvalues/vectors
    eigenvalues_v, eigenvectors_v = np.linalg.eigh(L0)
    eigenvalues_e, eigenvectors_e = np.linalg.eigh(L1)
    
    # average clustering coefficient/shortest path lengths of nodes
    average_clustering = nx.average_clustering(G)
    average_shortest_path_length = nx.average_shortest_path_length(G)
    
    X_dict = nx.kamada_kawai_layout(G)
    X = np.zeros((n,2))
    for j in X_dict:
        X[j] = X_dict[j]
    
    ax[i].set_title("p={}, $\Gamma$={:.2f}, $\Pi$={:.2f}".format(p_i, average_clustering, average_shortest_path_length))
    ax[i].scatter(X[:,0], X[:,1], c="k", s=10)

    # plot edges
    for e in G.edges():
        (start_id, end_id) = e
        ax[i].plot([X[start_id,0], X[end_id,0]], [X[start_id,1], X[end_id,1]], 'c-', alpha=0.5)
        
    # plot triangles
    for triangle in K[2]:
        t = plt.Polygon(stack(triangle), color="blue", alpha=0.02)
        ax[i].add_patch(t)

plt.show()

#### Hodge Laplacian spectra

In [ ]:
N = 100

spectra0 = []
spectra012 = []
spectra1 = []
spectra0_norm = []
spectra012_norm = []
spectra1_norm = []

for p_i in tqdm(np.repeat(ps, N)):
    
    # generate graph
    G = nx.generators.random_graphs.watts_strogatz_graph(n, m, p_i)
    
    # clique complex of graph
    K = expand_incremental(G, k=2)
    
    # boundary matrices
    B1 = boundary_matrix(K, k=1)
    B2 = boundary_matrix(K, k=2)
    B22 = generalized_boundary_matrix(K, k=2, p=2)
    
    # Laplacians
    L0 = B1 @ B1.T
    L1 = B1.T @ B1 + B2 @ B2.T
    L012 = B22 @ B22.T
    
    # normalized Laplacians
    L0_norm = normalize(L0)
    L1_norm = normalize(L1)
    L012_norm = normalize(L012)
    
    # eigenvalues
    eigenvalues_v, _ = np.linalg.eigh(L0)
    eigenvalues_e, _ = np.linalg.eigh(L1)
    eigenvalues_vt, _ = np.linalg.eigh(L012)
    eigenvalues_v_norm = np.sort(np.linalg.eigvals(L0_norm))
    eigenvalues_e_norm = np.sort(np.linalg.eigvals(L1_norm))
    eigenvalues_vt_norm = np.sort(np.linalg.eigvals(L012_norm))
    
    spectra0.append(eigenvalues_v)
    spectra1.append(eigenvalues_e)
    spectra012.append(eigenvalues_vt)
    spectra0_norm.append(eigenvalues_v_norm)
    spectra1_norm.append(eigenvalues_e_norm)
    spectra012_norm.append(eigenvalues_vt_norm)
    
spectra0 = np.array(spectra0)
spectra1 = np.array(spectra1)
spectra012 = np.array(spectra012)
spectra0_norm = np.array(spectra0_norm)
spectra1_norm = np.array(spectra1_norm)
spectra012_norm = np.array(spectra012_norm)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(16.5,5), dpi=200)
ax[0].set_title("$L_0$ spectrum")
ax[1].set_title("$L_{1}$ spectrum")
ax[2].set_title("$L_{012}$ spectrum")
ax[0].plot(spectra0[0:N].T, c="r", alpha=0.33)
ax[0].plot(spectra0[N:N*2].T, c="g", alpha=0.4)
ax[0].plot(spectra0[N*2:N*3].T, c="b", alpha=0.1)
ax[1].plot(spectra1[0:N].T, c="r", alpha=0.33)
ax[1].plot(spectra1[N:2*N].T, c="g", alpha=0.4)
ax[1].plot(spectra1[N*2:N*3].T, c="b", alpha=0.1)
ax[2].plot(spectra012[0:N].T, c="r", alpha=0.33)
ax[2].plot(spectra012[N:2*N].T, c="g", alpha=0.4)
ax[2].plot(spectra012[N*2:N*3].T, c="b", alpha=0.1)
plt.show()

### Classification

In [ ]:
n_repeats = 10
y = np.concatenate((np.ones(N) * 0, np.ones(N) * 1, np.ones(N) * 2))

#### L0

In [ ]:
%%time
results = []
for r in range(n_repeats):
    clf = RandomForestClassifier(random_state=r)
    results.append(list(cross_val_score(clf, spectra0, y, cv=5)))
    
results = np.array(results)
print("{:.4f}".format(np.mean(results) * 100))

#### L1

In [ ]:
%%time
results = []
for r in range(n_repeats):
    clf = RandomForestClassifier(random_state=r)
    results.append(list(cross_val_score(clf, spectra1, y, cv=5)))
    
results = np.array(results)
print("{:.4f}".format(np.mean(results) * 100))

#### L012

In [ ]:
%%time
results = []
for r in range(n_repeats):
    clf = RandomForestClassifier(random_state=r)
    results.append(list(cross_val_score(clf, spectra012, y, cv=5)))
    
results = np.array(results)
print("{:.4f}".format(np.mean(results) * 100))